### Dependencies

In [1]:
def get_f_period(p_date,f_date):
    d = pd.to_datetime(p_date) - pd.to_datetime(f_date)
    return d.days * 24  + d.seconds // 3600

import os 
import pickle
import pandas as pd
import numpy as np

### Load Data 
- Last Measurements (from 01/03/2021)
- Last 5 months of Forecasts

In [2]:
# Measurements
measurement_out = pd.read_csv('../data/processed/last_measurement_21102021.csv')
measurement_out['datetime'] = measurement_out['datetime'].map(lambda x : pd.to_datetime(x)) 
# Forecasts
forecast = pd.read_csv('../data/processed/last_forecast_21102021.csv')
forecast['f_date'] = forecast['f_date'].map(lambda x : pd.to_datetime(x))
forecast['p_date'] = forecast['p_date'].map(lambda x : pd.to_datetime(x))
forecast['file_creation_date'] = forecast['file_creation_date'].map(lambda x : pd.to_datetime(x))

### Data Processing
- Shift measurements to get 49 lag
- Data Merge 
- Keep last forecasts

In [3]:
### Data Merge ###

# Save a copy of measurements to score results
Y_real = measurement_out.copy()

# 49 lag of measurements horizontal stack 
df_out = Y_real.add_suffix('_t-0')
for i in range(1, 49):
    df_temp = Y_real.copy().add_suffix('_t-'+str(i))
    df_out = pd.concat([df_out,df_temp.shift(i)],axis=1)
df_out = df_out.dropna(how='any')
#display(df_out.head(1))

# join measurements & forecast
df_joined = df_out.copy()
df_joined = df_joined.merge(forecast.add_suffix('_forecast'),
                 how='left',
                 left_on = 'datetime_t-0',
                 right_on='f_date_forecast')

# filter forecast files created after prediction time (same as crop out f_period > 7)
df_joined = df_joined.loc[df_joined['datetime_t-0'] >= df_joined['file_creation_date_forecast'],]


# Compute f_period
df_joined['f_period'] = df_joined[['datetime_t-0','p_date_forecast']] \
                         .apply(lambda row : get_f_period(row['datetime_t-0'],row['p_date_forecast']),axis=1)

# assert that file_creation_date_forecast is doing the job
assert((df_joined.f_period > 7).any()) 

# keep last forecast
df_joined = df_joined.groupby('datetime_t-0')['f_period'].min().reset_index() \
             .merge(df_joined,how='left')
    
# compute cos day and hour 
df_joined['cos_day'] = np.cos(2 * np.pi * df_joined['datetime_t-0'].dt.day / 365)
df_joined['cos_hour'] =  np.cos(2 * np.pi * df_joined['datetime_t-0'].dt.hour / 24)
#display(df_joined.head(1))

### New models adjustements
- Compute forecast scenario & Dangerous
- Rename f_period -> f_period_forecast

In [5]:
# Compute needed columns for updated models
def get_int_scenario(speed, cos, sin):
    if is_S1(speed, cos, sin):
        return 1
    elif is_S2(speed, cos, sin):
        return 2
    elif is_S2b(speed, cos, sin):
        return 3
    elif is_S3(speed, cos, sin):
        return 4
    elif is_S3b(speed, cos, sin):
        return 5
    elif is_S4(speed, cos, sin):
        return 6
    return np.nan

import sys
sys.path.append('../')
from utils.util_functions import *

df_joined['scenario_forecast'] = df_joined.apply(lambda row : get_int_scenario(row['speed_forecast'],
                                             row['cos_wind_dir_forecast'],
                                             row['sin_wind_dir_forecast']),
                  axis=1)

df_joined['dangerous_forecast'] = (df_joined['scenario_forecast'] > 3 ).map(int)

df_joined = df_joined.rename(columns={'f_period':'f_period_forecast'})

### Make predictions
- Load all models (regression & classification
- Merge predictions : model_feature_lag (prediction date is shifted to align with measurement t-0)

In [9]:
import xgboost
xgboost.__version__

'1.1.1'

In [11]:
'../models_09072021/trained_models/' + x + '.pkl'

'../models_09072021/trained_models/xgb_speed_1.pkl'

In [15]:
# To render results on app we need a branch here for datetime shift ###
from datetime import timedelta

# Load needed columns for all models 
columns_names = list(pd.read_csv('../models_09072021/column_names.csv')['0'])

#model_names = ['xgb', 'dt','mlp','rf']
model_names = ['dt','mlp','rf']
features = ['speed','cos_wind_dir','sin_wind_dir','scenario','dangerous']
pred_periods = ['1','2','3']


models = dict()
for model_name in model_names:
    print(model_name)
    for feature in features:
        for pred_period in pred_periods:
            x = '_'.join([model_name,feature,pred_period])
            # Load model
            models[x] = pickle.load(open('../models_09072021/trained_models/' + x + '.pkl','rb'))
            df_temp = df_joined[['datetime_t-0']].copy()
            # Shift date
            df_temp['datetime_t-0'] = df_temp['datetime_t-0'] + timedelta(hours=int(pred_period))
            # Predict
            df_temp[x] = models[x].predict(df_joined[columns_names])
            # Save
            df_joined = df_joined.merge(df_temp,how='left')
            del df_temp

# Drop missing values (at least the 3 first rows)
df_joined = df_joined.dropna()
df_joined.shape

dt


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking co

mlp


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your o

rf


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.22.1. This might lead to breaking co

(469, 452)

### Speed regression mae

In [16]:
from sklearn.metrics import mean_absolute_error

### Speed Mae ###
print('Model   1   2   3')
for model in ['rf','dt','mlp']:
    print(model,
          round(mean_absolute_error(df_joined['speed_t-0'],df_joined[model + '_speed_1']),2),
          round(mean_absolute_error(df_joined['speed_t-0'],df_joined[model + '_speed_2']),2),
          round(mean_absolute_error(df_joined['speed_t-0'],df_joined[model + '_speed_3']),2)
         )
print('Numtech' , round(mean_absolute_error(df_joined['speed_t-0'],df_joined['speed_forecast']),2))

Model   1   2   3
rf 0.52 0.75 0.93
dt 0.54 0.75 1.0
mlp 0.73 1.21 1.16
Numtech 0.91


### Wind Direction Regression Mae

In [17]:
def get_angle_in_degree(cos, sin):
    #check if cos within reasonable range: 
    if (cos>=-1) & (cos <=1): 
        angle = 360 * np.arccos(cos) / (2*np.pi)
        if sin < 0:
            angle = 360 - angle
    #check if sin within reasonable range:       
    elif (sin>=-1) & (sin <=1):
        angle = 360 * np.arcsin(sin) / (2*np.pi)
        if cos < 0:
            angle = 180 - angle
        if angle < 0:
            angle += 360
    else:
        angle=0 
        #print('cos and sin out of range, returned 0')
    #because we care about the reverse angle for the scenarios
    return angle #(angle + 180) % 360

def angle_diff(x):
    if x <= 180:
        return x 
    else:
        return 360-180
    

### Real wind dir ###
df_joined['wind_dir_t-0'] =  df_joined.apply(lambda row : get_angle_in_degree(row['cos_wind_dir_t-0'],
                                                                              row['sin_wind_dir_t-0']),
                                             axis=1)
    
for model_name in model_names:
    for pred_period in pred_periods:
        input_cos = model_name + '_cos_wind_dir_' + pred_period
        input_sin = model_name + '_sin_wind_dir_' + pred_period
        output_wind_dir = model_name + '_wind_dir_' + pred_period
        df_joined[output_wind_dir] =  df_joined.apply(lambda row : get_angle_in_degree(row[input_cos],row[input_sin]),axis=1)
        
print('Model  1  2  3')
for model_name in model_names:
    print(model_name,
          round((df_joined[model_name + '_wind_dir_1'] - df_joined['wind_dir_t-0']).abs().map(angle_diff).mean()),
          round((df_joined[model_name + '_wind_dir_2'] - df_joined['wind_dir_t-0']).abs().map(angle_diff).mean()),
          round((df_joined[model_name + '_wind_dir_3'] - df_joined['wind_dir_t-0']).abs().map(angle_diff).mean())
         )
print("Numtech ",round((df_joined['wind_dir_forecast'] - df_joined['wind_dir_t-0']).abs().map(angle_diff).mean()))

Model  1  2  3
dt 20 35 41
mlp 26 48 45
rf 19 31 43
Numtech  43


### Confusion Matrix 

In [19]:
# Real scenario & dangerous #
df_joined['scenario_t-0'] = df_joined.apply(lambda row : get_int_scenario(row['speed_t-0'],
                                                                          row['cos_wind_dir_t-0'],
                                                                          row['sin_wind_dir_t-0']),
                                                    axis=1)
df_joined['dangerous_t-0'] = (df_joined['scenario_t-0'] > 3).map(int)


# Regression scenario & dangerous
for model_name in model_names:
    for pred_period in pred_periods:
        
        input_speed =  model_name + '_speed_' + pred_period
        input_cos = model_name + '_cos_wind_dir_' + pred_period
        input_sin = model_name + '_sin_wind_dir_' + pred_period
        output_scenario = model_name + '_regression_scenario_' + pred_period
        output_dangerous = model_name + '_regression_dangerous_' + pred_period
        
        df_joined[output_scenario] = df_joined.apply(lambda row : get_int_scenario(row[input_speed],
                                                                                   row[input_cos],
                                                                                   row[input_sin]),
                                                    axis=1)
        df_joined[output_dangerous] = (df_joined[output_scenario] > 3).map(int)
        
    
from sklearn.metrics import confusion_matrix

df_result = []

for model_name in model_names:
    #print(model_name)
    for pred_period in pred_periods:
        # regression
        cm = confusion_matrix(df_joined['dangerous_t-0'],df_joined[model_name + '_regression_dangerous_' + pred_period])
        df_result += [['regression',model_name,pred_period,cm[0][0],cm[0][1],cm[1][0],cm[1][1]]]
        # scenario
        df_joined[model_name + '_sc_dangerous_' + pred_period] = (df_joined[model_name + '_scenario_' + pred_period] > 3).map(int)
        cm = confusion_matrix(df_joined['dangerous_t-0'],df_joined[model_name + '_sc_dangerous_' + pred_period])
        df_result += [['sc classification',model_name,pred_period,cm[0][0],cm[0][1],cm[1][0],cm[1][1]]]
        # binary
        cm = confusion_matrix(df_joined['dangerous_t-0'],df_joined[model_name + '_dangerous_' + pred_period])
        df_result += [['binary classification',model_name,pred_period,cm[0][0],cm[0][1],cm[1][0],cm[1][1]]]
        
        
cm = confusion_matrix(df_joined['dangerous_t-0'],df_joined['dangerous_forecast'])
df_result += [['','Numtech','',cm[0][0],cm[0][1],cm[1][0],cm[1][1]]]

df_result = pd.DataFrame(df_result,columns=['Type','Name','Period','Correct Safe','Wrong Dangerous','Wrong Safe','Correct Dangerous'])

df_result.to_excel('./results.xlsx',index=False)

df_result.sort_values(by=['Type','Name','Period'])

,Type,Name,Period,Correct Safe,Wrong Dangerous,Wrong Safe,Correct Dangerous
27,,Numtech,,420,30,11,8
2,binary classification,dt,1,416,34,10,9
5,binary classification,dt,2,441,9,18,1
8,binary classification,dt,3,413,37,11,8
11,binary classification,mlp,1,450,0,18,1
14,binary classification,mlp,2,450,0,18,1
17,binary classification,mlp,3,449,1,19,0
20,binary classification,rf,1,405,45,2,17
23,binary classification,rf,2,391,59,6,13
26,binary classification,rf,3,388,62,7,12


In [9]:
df_joined

,datetime_t-0,f_period_forecast,speed_t-0,cos_wind_dir_t-0,sin_wind_dir_t-0,temp_t-0,radiation_t-0,precip_t-0,season_t-0,datetime_t-1,...,xgb_sc_dangerous_3,dt_sc_dangerous_1,dt_sc_dangerous_2,dt_sc_dangerous_3,mlp_sc_dangerous_1,mlp_sc_dangerous_2,mlp_sc_dangerous_3,rf_sc_dangerous_1,rf_sc_dangerous_2,rf_sc_dangerous_3
3,2021-03-04 23:00:00,11,3.298333,-0.427917,-0.883472,17.795000,0.000000,0.000000,2,2021-03-04 22:00:00,...,0,0,0,0,0,0,0,0,0,0
4,2021-03-05 00:00:00,12,3.570000,-0.548271,-0.808114,17.878333,0.000000,0.000000,2,2021-03-04 23:00:00,...,0,0,0,0,0,0,0,0,0,0
5,2021-03-05 01:00:00,13,5.520000,-0.324955,-0.901145,17.111667,0.000000,0.003333,2,2021-03-05 00:00:00,...,0,0,0,0,0,0,0,0,0,0
6,2021-03-05 02:00:00,14,2.786667,0.112619,-0.923949,15.781667,0.000000,0.000000,2,2021-03-05 01:00:00,...,0,0,0,0,0,0,0,0,0,0
7,2021-03-05 03:00:00,15,2.290000,-0.704985,-0.580373,16.040000,0.000000,0.000000,2,2021-03-05 02:00:00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,2021-07-28 11:00:00,11,3.661667,0.688993,-0.261954,27.501667,573.666667,0.000000,3,2021-07-28 10:00:00,...,0,0,0,0,0,0,0,0,0,0
3456,2021-07-28 12:00:00,12,5.678333,0.728029,-0.633090,25.698333,640.500000,0.000000,3,2021-07-28 11:00:00,...,0,0,0,0,0,0,0,0,0,0
3457,2021-07-28 13:00:00,13,7.738333,0.732002,-0.642845,25.006667,669.066667,0.000000,3,2021-07-28 12:00:00,...,0,0,0,0,0,0,0,0,0,0
3458,2021-07-28 14:00:00,14,7.436667,0.727829,-0.665280,25.268333,654.283333,0.000000,3,2021-07-28 13:00:00,...,0,0,0,0,0,0,0,0,0,0
